In [1]:
%matplotlib inline

In general both transfer learning methods follow the same few steps:

-  Initialize the pretrained model
-  Reshape the final layer(s) to have the same number of outputs as the
   number of classes in the new dataset
-  Define for the optimization algorithm which parameters we want to
   update during training
-  Run the training step

In [2]:
#from __future__ import print_function 
#from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.11.0
Torchvision Version:  0.12.0


In [3]:

from torch.utils.data import DataLoader

from dvmcar import DvmCarDataset

# Partition dataset into train, test, and validate subsets
scale = 0.10
partition0  = 0.8*scale
partition1  = 0.9*scale
partition2  = 1.0*scale

train_split = [0,          partition0]
val_split   = [partition0, partition1]
test_split  = [partition1, partition2]

batch_size  = 64
shuffle     = True
input_size  = 224

train_transform = transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
val_transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])

train_data  = DvmCarDataset(split = train_split, transform = train_transform)
val_data    = DvmCarDataset(split =   val_split, transform =   val_transform)

print(len(train_data))
print(len(val_data))


Work file /data/dvmcar/dvmcar.zip is already available.
Using existing /data/dvmcar\resized_DVM_v2.zip.
Using existing /data/dvmcar\Confirmed_fronts.zip.
Using existing /data/dvmcar\tables_V2.0.zip.
Work file /data/dvmcar/dvmcar.zip is already available.
Using existing /data/dvmcar\resized_DVM_v2.zip.
Using existing /data/dvmcar\Confirmed_fronts.zip.
Using existing /data/dvmcar\tables_V2.0.zip.
116142
14518


Inputs
------

Here are all of the parameters to change for the run. We will use the
*hymenoptera_data* dataset which can be downloaded
`here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`__.
This dataset contains two classes, **bees** and **ants**, and is
structured such that we can use the
`ImageFolder <https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.ImageFolder>`__
dataset, rather than writing our own custom dataset. Download the data
and set the ``data_dir`` input to the root directory of the dataset. The
``model_name`` input is the name of the model you wish to use and must
be selected from this list:

::

   [resnet, alexnet, vgg, squeezenet, densenet, inception]

The other inputs are as follows: ``num_classes`` is the number of
classes in the dataset, ``batch_size`` is the batch size used for
training and may be adjusted according to the capability of your
machine, ``num_epochs`` is the number of training epochs we want to run,
and ``feature_extract`` is a boolean that defines if we are finetuning
or feature extracting. If ``feature_extract = False``, the model is
finetuned and all model parameters are updated. If
``feature_extract = True``, only the last layer parameters are updated,
the others remain fixed.




In [4]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = train_data.classes

# Batch size for training (change depending on how much memory you have)
batch_size = 64

# Number of epochs to train for 
num_epochs = 100

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = True

Helper Functions
----------------

Before we write the code for adjusting the models, lets define a few
helper functions.

Model Training and Validation Code
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

The ``train_model`` function handles the training and validation of a
given model. As input, it takes a PyTorch model, a dictionary of
dataloaders, a loss function, an optimizer, a specified number of epochs
to train and validate for, and a boolean flag for when the model is an
Inception model. The *is_inception* flag is used to accomodate the
*Inception v3* model, as that architecture uses an auxiliary output and
the overall model loss respects both the auxiliary output and the final
output, as described
`here <https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958>`__.
The function trains for the specified number of epochs and after each
epoch runs a full validation step. It also keeps track of the best
performing model (in terms of validation accuracy), and at the end of
training returns the best performing model. After each epoch, the
training and validation accuracies are printed.




In [5]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            sample = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                print('Epoch{0} / Sample {1}'.format(epoch,sample))

                sample += inputs.size(0)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

Set Model Parameters’ .requires_grad attribute
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

This helper function sets the ``.requires_grad`` attribute of the
parameters in the model to False when we are feature extracting. By
default, when we load a pretrained model all of the parameters have
``.requires_grad=True``, which is fine if we are training from scratch
or finetuning. However, if we are feature extracting and only want to
compute gradients for the newly initialized layer then we want all of
the other parameters to not require gradients. This will make more sense
later.




In [6]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

Initialize and Reshape the Networks
-----------------------------------

Now to the most interesting part. Here is where we handle the reshaping
of each network. Note, this is not an automatic procedure and is unique
to each model. Recall, the final layer of a CNN model, which is often
times an FC layer, has the same number of nodes as the number of output
classes in the dataset. Since all of the models have been pretrained on
Imagenet, they all have output layers of size 1000, one node for each
class. The goal here is to reshape the last layer to have the same
number of inputs as before, AND to have the same number of outputs as
the number of classes in the dataset. In the following sections we will
discuss how to alter the architecture of each model individually. But
first, there is one important detail regarding the difference between
finetuning and feature-extraction.

When feature extracting, we only want to update the parameters of the
last layer, or in other words, we only want to update the parameters for
the layer(s) we are reshaping. Therefore, we do not need to compute the
gradients of the parameters that we are not changing, so for efficiency
we set the .requires_grad attribute to False. This is important because
by default, this attribute is set to True. Then, when we initialize the
new layer and by default the new parameters have ``.requires_grad=True``
so only the new layer’s parameters will be updated. When we are
finetuning we can leave all of the .required_grad’s set to the default
of True.

Finally, notice that inception_v3 requires the input size to be
(299,299), whereas all of the other models expect (224,224).

Resnet
~~~~~~

Resnet was introduced in the paper `Deep Residual Learning for Image
Recognition <https://arxiv.org/abs/1512.03385>`__. There are several
variants of different sizes, including Resnet18, Resnet34, Resnet50,
Resnet101, and Resnet152, all of which are available from torchvision
models. Here we use Resnet18, as our dataset is small and only has two
classes. When we print the model, we see that the last layer is a fully
connected layer as shown below:

::

   (fc): Linear(in_features=512, out_features=1000, bias=True) 

Thus, we must reinitialize ``model.fc`` to be a Linear layer with 512
input features and 2 output features with:

::

   model.fc = nn.Linear(512, num_classes)

Alexnet
~~~~~~~

Alexnet was introduced in the paper `ImageNet Classification with Deep
Convolutional Neural
Networks <https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf>`__
and was the first very successful CNN on the ImageNet dataset. When we
print the model architecture, we see the model output comes from the 6th
layer of the classifier

::

   (classifier): Sequential(
       ...
       (6): Linear(in_features=4096, out_features=1000, bias=True)
    ) 

To use the model with our dataset we reinitialize this layer as

::

   model.classifier[6] = nn.Linear(4096,num_classes)

VGG
~~~

VGG was introduced in the paper `Very Deep Convolutional Networks for
Large-Scale Image Recognition <https://arxiv.org/pdf/1409.1556.pdf>`__.
Torchvision offers eight versions of VGG with various lengths and some
that have batch normalizations layers. Here we use VGG-11 with batch
normalization. The output layer is similar to Alexnet, i.e.

::

   (classifier): Sequential(
       ...
       (6): Linear(in_features=4096, out_features=1000, bias=True)
    )

Therefore, we use the same technique to modify the output layer

::

   model.classifier[6] = nn.Linear(4096,num_classes)

Squeezenet
~~~~~~~~~~

The Squeeznet architecture is described in the paper `SqueezeNet:
AlexNet-level accuracy with 50x fewer parameters and <0.5MB model
size <https://arxiv.org/abs/1602.07360>`__ and uses a different output
structure than any of the other models shown here. Torchvision has two
versions of Squeezenet, we use version 1.0. The output comes from a 1x1
convolutional layer which is the 1st layer of the classifier:

::

   (classifier): Sequential(
       (0): Dropout(p=0.5)
       (1): Conv2d(512, 1000, kernel_size=(1, 1), stride=(1, 1))
       (2): ReLU(inplace)
       (3): AvgPool2d(kernel_size=13, stride=1, padding=0)
    ) 

To modify the network, we reinitialize the Conv2d layer to have an
output feature map of depth 2 as

::

   model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))

Densenet
~~~~~~~~

Densenet was introduced in the paper `Densely Connected Convolutional
Networks <https://arxiv.org/abs/1608.06993>`__. Torchvision has four
variants of Densenet but here we only use Densenet-121. The output layer
is a linear layer with 1024 input features:

::

   (classifier): Linear(in_features=1024, out_features=1000, bias=True) 

To reshape the network, we reinitialize the classifier’s linear layer as

::

   model.classifier = nn.Linear(1024, num_classes)

Inception v3
~~~~~~~~~~~~

Finally, Inception v3 was first described in `Rethinking the Inception
Architecture for Computer
Vision <https://arxiv.org/pdf/1512.00567v1.pdf>`__. This network is
unique because it has two output layers when training. The second output
is known as an auxiliary output and is contained in the AuxLogits part
of the network. The primary output is a linear layer at the end of the
network. Note, when testing we only consider the primary output. The
auxiliary output and primary output of the loaded model are printed as:

::

   (AuxLogits): InceptionAux(
       ...
       (fc): Linear(in_features=768, out_features=1000, bias=True)
    )
    ...
   (fc): Linear(in_features=2048, out_features=1000, bias=True)

To finetune this model we must reshape both layers. This is accomplished
with the following

::

   model.AuxLogits.fc = nn.Linear(768, num_classes)
   model.fc = nn.Linear(2048, num_classes)

Notice, many of the models have similar output structures, but each must
be handled slightly differently. Also, check out the printed model
architecture of the reshaped network and make sure the number of output
features is the same as the number of classes in the dataset.




In [7]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Load Data
---------

Now that we know what the input size must be, we can initialize the data
transforms, image datasets, and the dataloaders. Notice, the models were
pretrained with the hard-coded normalization values, as described
`here <https://pytorch.org/docs/master/torchvision/models.html>`__.




In [8]:


print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {'train': train_data, 'val' : val_data}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


Create the Optimizer
--------------------

Now that the model structure is correct, the final step for finetuning
and feature extracting is to create an optimizer that only updates the
desired parameters. Recall that after loading the pretrained model, but
before reshaping, if ``feature_extract=True`` we manually set all of the
parameter’s ``.requires_grad`` attributes to False. Then the
reinitialized layer’s parameters have ``.requires_grad=True`` by
default. So now we know that *all parameters that have
.requires_grad=True should be optimized.* Next, we make a list of such
parameters and input this list to the SGD algorithm constructor.

To verify this, check out the printed parameters to learn. When
finetuning, this list should be long and include all of the model
parameters. However, when feature extracting this list should be short
and only include the weights and biases of the reshaped layers.




In [9]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


Run Training and Validation Step
--------------------------------

Finally, the last step is to setup the loss for the model, then run the
training and validation function for the set number of epochs. Notice,
depending on the number of epochs this step may take a while on a CPU.
Also, the default learning rate is not optimal for all of the models, so
to achieve maximum accuracy it would be necessary to tune for each model
separately.




In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/99
----------
Epoch0 / Sample 0
Epoch0 / Sample 64
Epoch0 / Sample 128
Epoch0 / Sample 192
Epoch0 / Sample 256
Epoch0 / Sample 320
Epoch0 / Sample 384
Epoch0 / Sample 448
Epoch0 / Sample 512
Epoch0 / Sample 576
Epoch0 / Sample 640
Epoch0 / Sample 704
Epoch0 / Sample 768
Epoch0 / Sample 832
Epoch0 / Sample 896
Epoch0 / Sample 960
Epoch0 / Sample 1024
Epoch0 / Sample 1088
Epoch0 / Sample 1152
Epoch0 / Sample 1216
Epoch0 / Sample 1280
Epoch0 / Sample 1344
Epoch0 / Sample 1408
Epoch0 / Sample 1472
Epoch0 / Sample 1536
Epoch0 / Sample 1600
Epoch0 / Sample 1664
Epoch0 / Sample 1728
Epoch0 / Sample 1792
Epoch0 / Sample 1856
Epoch0 / Sample 1920
Epoch0 / Sample 1984
Epoch0 / Sample 2048
Epoch0 / Sample 2112
Epoch0 / Sample 2176
Epoch0 / Sample 2240
Epoch0 / Sample 2304
Epoch0 / Sample 2368
Epoch0 / Sample 2432
Epoch0 / Sample 2496
Epoch0 / Sample 2560
Epoch0 / Sample 2624
Epoch0 / Sample 2688
Epoch0 / Sample 2752
Epoch0 / Sample 2816
Epoch0 / Sample 2880
Epoch0 / Sample 2944
Epoch0 / S

Epoch0 / Sample 24320
Epoch0 / Sample 24384
Epoch0 / Sample 24448
Epoch0 / Sample 24512
Epoch0 / Sample 24576
Epoch0 / Sample 24640
Epoch0 / Sample 24704
Epoch0 / Sample 24768
Epoch0 / Sample 24832
Epoch0 / Sample 24896
Epoch0 / Sample 24960
Epoch0 / Sample 25024
Epoch0 / Sample 25088
Epoch0 / Sample 25152
Epoch0 / Sample 25216
Epoch0 / Sample 25280
Epoch0 / Sample 25344
Epoch0 / Sample 25408
Epoch0 / Sample 25472
Epoch0 / Sample 25536
Epoch0 / Sample 25600
Epoch0 / Sample 25664
Epoch0 / Sample 25728
Epoch0 / Sample 25792
Epoch0 / Sample 25856
Epoch0 / Sample 25920
Epoch0 / Sample 25984
Epoch0 / Sample 26048
Epoch0 / Sample 26112
Epoch0 / Sample 26176
Epoch0 / Sample 26240
Epoch0 / Sample 26304
Epoch0 / Sample 26368
Epoch0 / Sample 26432
Epoch0 / Sample 26496
Epoch0 / Sample 26560
Epoch0 / Sample 26624
Epoch0 / Sample 26688
Epoch0 / Sample 26752
Epoch0 / Sample 26816
Epoch0 / Sample 26880
Epoch0 / Sample 26944
Epoch0 / Sample 27008
Epoch0 / Sample 27072
Epoch0 / Sample 27136
Epoch0 / S

Epoch0 / Sample 48192
Epoch0 / Sample 48256
Epoch0 / Sample 48320
Epoch0 / Sample 48384
Epoch0 / Sample 48448
Epoch0 / Sample 48512
Epoch0 / Sample 48576
Epoch0 / Sample 48640
Epoch0 / Sample 48704
Epoch0 / Sample 48768
Epoch0 / Sample 48832
Epoch0 / Sample 48896
Epoch0 / Sample 48960
Epoch0 / Sample 49024
Epoch0 / Sample 49088
Epoch0 / Sample 49152
Epoch0 / Sample 49216
Epoch0 / Sample 49280
Epoch0 / Sample 49344
Epoch0 / Sample 49408
Epoch0 / Sample 49472
Epoch0 / Sample 49536
Epoch0 / Sample 49600
Epoch0 / Sample 49664
Epoch0 / Sample 49728
Epoch0 / Sample 49792
Epoch0 / Sample 49856
Epoch0 / Sample 49920
Epoch0 / Sample 49984
Epoch0 / Sample 50048
Epoch0 / Sample 50112
Epoch0 / Sample 50176
Epoch0 / Sample 50240
Epoch0 / Sample 50304
Epoch0 / Sample 50368
Epoch0 / Sample 50432
Epoch0 / Sample 50496
Epoch0 / Sample 50560
Epoch0 / Sample 50624
Epoch0 / Sample 50688
Epoch0 / Sample 50752
Epoch0 / Sample 50816
Epoch0 / Sample 50880
Epoch0 / Sample 50944
Epoch0 / Sample 51008
Epoch0 / S

Epoch0 / Sample 72064
Epoch0 / Sample 72128
Epoch0 / Sample 72192
Epoch0 / Sample 72256
Epoch0 / Sample 72320
Epoch0 / Sample 72384
Epoch0 / Sample 72448
Epoch0 / Sample 72512
Epoch0 / Sample 72576
Epoch0 / Sample 72640
Epoch0 / Sample 72704
Epoch0 / Sample 72768
Epoch0 / Sample 72832
Epoch0 / Sample 72896
Epoch0 / Sample 72960
Epoch0 / Sample 73024
Epoch0 / Sample 73088
Epoch0 / Sample 73152
Epoch0 / Sample 73216
Epoch0 / Sample 73280
Epoch0 / Sample 73344
Epoch0 / Sample 73408
Epoch0 / Sample 73472
Epoch0 / Sample 73536
Epoch0 / Sample 73600
Epoch0 / Sample 73664
Epoch0 / Sample 73728
Epoch0 / Sample 73792
Epoch0 / Sample 73856
Epoch0 / Sample 73920
Epoch0 / Sample 73984
Epoch0 / Sample 74048
Epoch0 / Sample 74112
Epoch0 / Sample 74176
Epoch0 / Sample 74240
Epoch0 / Sample 74304
Epoch0 / Sample 74368
Epoch0 / Sample 74432
Epoch0 / Sample 74496
Epoch0 / Sample 74560
Epoch0 / Sample 74624
Epoch0 / Sample 74688
Epoch0 / Sample 74752
Epoch0 / Sample 74816
Epoch0 / Sample 74880
Epoch0 / S

Epoch0 / Sample 95936
Epoch0 / Sample 96000
Epoch0 / Sample 96064
Epoch0 / Sample 96128
Epoch0 / Sample 96192
Epoch0 / Sample 96256
Epoch0 / Sample 96320
Epoch0 / Sample 96384
Epoch0 / Sample 96448
Epoch0 / Sample 96512
Epoch0 / Sample 96576
Epoch0 / Sample 96640
Epoch0 / Sample 96704
Epoch0 / Sample 96768
Epoch0 / Sample 96832
Epoch0 / Sample 96896
Epoch0 / Sample 96960
Epoch0 / Sample 97024
Epoch0 / Sample 97088
Epoch0 / Sample 97152
Epoch0 / Sample 97216
Epoch0 / Sample 97280
Epoch0 / Sample 97344
Epoch0 / Sample 97408
Epoch0 / Sample 97472
Epoch0 / Sample 97536
Epoch0 / Sample 97600
Epoch0 / Sample 97664
Epoch0 / Sample 97728
Epoch0 / Sample 97792
Epoch0 / Sample 97856
Epoch0 / Sample 97920
Epoch0 / Sample 97984
Epoch0 / Sample 98048
Epoch0 / Sample 98112
Epoch0 / Sample 98176
Epoch0 / Sample 98240
Epoch0 / Sample 98304
Epoch0 / Sample 98368
Epoch0 / Sample 98432
Epoch0 / Sample 98496
Epoch0 / Sample 98560
Epoch0 / Sample 98624
Epoch0 / Sample 98688
Epoch0 / Sample 98752
Epoch0 / S

...

Epoch3 / Sample 109376
Epoch3 / Sample 109440
Epoch3 / Sample 109504
Epoch3 / Sample 109568
Epoch3 / Sample 109632
Epoch3 / Sample 109696
Epoch3 / Sample 109760
Epoch3 / Sample 109824
Epoch3 / Sample 109888
Epoch3 / Sample 109952
Epoch3 / Sample 110016
Epoch3 / Sample 110080
Epoch3 / Sample 110144
Epoch3 / Sample 110208
Epoch3 / Sample 110272
Epoch3 / Sample 110336
Epoch3 / Sample 110400
Epoch3 / Sample 110464
Epoch3 / Sample 110528
Epoch3 / Sample 110592
Epoch3 / Sample 110656
Epoch3 / Sample 110720
Epoch3 / Sample 110784
Epoch3 / Sample 110848
Epoch3 / Sample 110912
Epoch3 / Sample 110976
Epoch3 / Sample 111040
Epoch3 / Sample 111104
Epoch3 / Sample 111168
Epoch3 / Sample 111232
Epoch3 / Sample 111296
Epoch3 / Sample 111360
Epoch3 / Sample 111424
Epoch3 / Sample 111488
Epoch3 / Sample 111552
Epoch3 / Sample 111616
Epoch3 / Sample 111680
Epoch3 / Sample 111744
Epoch3 / Sample 111808
Epoch3 / Sample 111872
Epoch3 / Sample 111936
Epoch3 / Sample 112000
Epoch3 / Sample 112064
Epoch3 / Sa

Epoch4 / Sample 2688
Epoch4 / Sample 2752
Epoch4 / Sample 2816
Epoch4 / Sample 2880
Epoch4 / Sample 2944
Epoch4 / Sample 3008
Epoch4 / Sample 3072
Epoch4 / Sample 3136
Epoch4 / Sample 3200
Epoch4 / Sample 3264
Epoch4 / Sample 3328
Epoch4 / Sample 3392
Epoch4 / Sample 3456
Epoch4 / Sample 3520
Epoch4 / Sample 3584
Epoch4 / Sample 3648
Epoch4 / Sample 3712
Epoch4 / Sample 3776
Epoch4 / Sample 3840
Epoch4 / Sample 3904
Epoch4 / Sample 3968
Epoch4 / Sample 4032
Epoch4 / Sample 4096
Epoch4 / Sample 4160
Epoch4 / Sample 4224
Epoch4 / Sample 4288
Epoch4 / Sample 4352
Epoch4 / Sample 4416
Epoch4 / Sample 4480
Epoch4 / Sample 4544
Epoch4 / Sample 4608
Epoch4 / Sample 4672
Epoch4 / Sample 4736
Epoch4 / Sample 4800
Epoch4 / Sample 4864
Epoch4 / Sample 4928
Epoch4 / Sample 4992
Epoch4 / Sample 5056
Epoch4 / Sample 5120
Epoch4 / Sample 5184
Epoch4 / Sample 5248
Epoch4 / Sample 5312
Epoch4 / Sample 5376
Epoch4 / Sample 5440
Epoch4 / Sample 5504
Epoch4 / Sample 5568
Epoch4 / Sample 5632
Epoch4 / Samp

Epoch4 / Sample 26880
Epoch4 / Sample 26944
Epoch4 / Sample 27008
Epoch4 / Sample 27072
Epoch4 / Sample 27136
Epoch4 / Sample 27200
Epoch4 / Sample 27264
Epoch4 / Sample 27328
Epoch4 / Sample 27392
Epoch4 / Sample 27456
Epoch4 / Sample 27520
Epoch4 / Sample 27584
Epoch4 / Sample 27648
Epoch4 / Sample 27712
Epoch4 / Sample 27776
Epoch4 / Sample 27840
Epoch4 / Sample 27904
Epoch4 / Sample 27968
Epoch4 / Sample 28032
Epoch4 / Sample 28096
Epoch4 / Sample 28160
Epoch4 / Sample 28224
Epoch4 / Sample 28288
Epoch4 / Sample 28352
Epoch4 / Sample 28416
Epoch4 / Sample 28480
Epoch4 / Sample 28544
Epoch4 / Sample 28608
Epoch4 / Sample 28672
Epoch4 / Sample 28736
Epoch4 / Sample 28800
Epoch4 / Sample 28864
Epoch4 / Sample 28928
Epoch4 / Sample 28992
Epoch4 / Sample 29056
Epoch4 / Sample 29120
Epoch4 / Sample 29184
Epoch4 / Sample 29248
Epoch4 / Sample 29312
Epoch4 / Sample 29376
Epoch4 / Sample 29440
Epoch4 / Sample 29504
Epoch4 / Sample 29568
Epoch4 / Sample 29632
Epoch4 / Sample 29696
Epoch4 / S

Epoch4 / Sample 50752
Epoch4 / Sample 50816
Epoch4 / Sample 50880
Epoch4 / Sample 50944
Epoch4 / Sample 51008
Epoch4 / Sample 51072
Epoch4 / Sample 51136
Epoch4 / Sample 51200
Epoch4 / Sample 51264
Epoch4 / Sample 51328
Epoch4 / Sample 51392
Epoch4 / Sample 51456
Epoch4 / Sample 51520
Epoch4 / Sample 51584
Epoch4 / Sample 51648
Epoch4 / Sample 51712
Epoch4 / Sample 51776
Epoch4 / Sample 51840
Epoch4 / Sample 51904
Epoch4 / Sample 51968
Epoch4 / Sample 52032
Epoch4 / Sample 52096
Epoch4 / Sample 52160
Epoch4 / Sample 52224
Epoch4 / Sample 52288
Epoch4 / Sample 52352
Epoch4 / Sample 52416
Epoch4 / Sample 52480
Epoch4 / Sample 52544
Epoch4 / Sample 52608
Epoch4 / Sample 52672
Epoch4 / Sample 52736
Epoch4 / Sample 52800
Epoch4 / Sample 52864
Epoch4 / Sample 52928
Epoch4 / Sample 52992
Epoch4 / Sample 53056
Epoch4 / Sample 53120
Epoch4 / Sample 53184
Epoch4 / Sample 53248
Epoch4 / Sample 53312
Epoch4 / Sample 53376
Epoch4 / Sample 53440
Epoch4 / Sample 53504
Epoch4 / Sample 53568
Epoch4 / S

Epoch4 / Sample 74624
Epoch4 / Sample 74688
Epoch4 / Sample 74752
Epoch4 / Sample 74816
Epoch4 / Sample 74880
Epoch4 / Sample 74944
Epoch4 / Sample 75008
Epoch4 / Sample 75072
Epoch4 / Sample 75136
Epoch4 / Sample 75200
Epoch4 / Sample 75264
Epoch4 / Sample 75328
Epoch4 / Sample 75392
Epoch4 / Sample 75456
Epoch4 / Sample 75520
Epoch4 / Sample 75584
Epoch4 / Sample 75648
Epoch4 / Sample 75712
Epoch4 / Sample 75776
Epoch4 / Sample 75840
Epoch4 / Sample 75904
Epoch4 / Sample 75968
Epoch4 / Sample 76032
Epoch4 / Sample 76096
Epoch4 / Sample 76160
Epoch4 / Sample 76224
Epoch4 / Sample 76288
Epoch4 / Sample 76352
Epoch4 / Sample 76416
Epoch4 / Sample 76480
Epoch4 / Sample 76544
Epoch4 / Sample 76608
Epoch4 / Sample 76672
Epoch4 / Sample 76736
Epoch4 / Sample 76800
Epoch4 / Sample 76864
Epoch4 / Sample 76928
Epoch4 / Sample 76992
Epoch4 / Sample 77056
Epoch4 / Sample 77120
Epoch4 / Sample 77184
Epoch4 / Sample 77248
Epoch4 / Sample 77312
Epoch4 / Sample 77376
Epoch4 / Sample 77440
Epoch4 / S

Comparison with Model Trained from Scratch
------------------------------------------

Just for fun, lets see how the model learns if we do not use transfer
learning. The performance of finetuning vs. feature extracting depends
largely on the dataset but in general both transfer learning methods
produce favorable results in terms of training time and overall accuracy
versus a model trained from scratch.




In [ ]:
# Initialize the non-pretrained version of the model used for this run
scratch_model,_ = initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=False)
scratch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
_,scratch_hist = train_model(scratch_model, dataloaders_dict, scratch_criterion, scratch_optimizer, num_epochs=num_epochs, is_inception=(model_name=="inception"))

# Plot the training curves of validation accuracy vs. number 
#  of training epochs for the transfer learning method and
#  the model trained from scratch
ohist = []
shist = []

ohist = [h.cpu().numpy() for h in hist]
shist = [h.cpu().numpy() for h in scratch_hist]

plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
plt.plot(range(1,num_epochs+1),shist,label="Scratch")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()

Final Thoughts and Where to Go Next
-----------------------------------

Try running some of the other models and see how good the accuracy gets.
Also, notice that feature extracting takes less time because in the
backward pass we do not have to calculate most of the gradients. There
are many places to go from here. You could:

-  Run this code with a harder dataset and see some more benefits of
   transfer learning
-  Using the methods described here, use transfer learning to update a
   different model, perhaps in a new domain (i.e. NLP, audio, etc.)
-  Once you are happy with a model, you can export it as an ONNX model,
   or trace it using the hybrid frontend for more speed and optimization
   opportunities.


